In [2]:
!pip install -q kaggle

In [3]:
!mkdir ~/.kaggle


In [5]:
!cp kaggle.json ~/.kaggle

In [6]:
!kaggle datasets download -d sritulasi/animal-classiffication

 92% 57.0M/62.3M [00:00<00:00, 124MB/s]
100% 62.3M/62.3M [00:00<00:00, 111MB/s]


In [7]:
!unzip /content/animal-classiffication.zip

Archive:  /content/animal-classiffication.zip
  inflating: dataset/Testing/bears/k4 (100).jpeg  
  inflating: dataset/Testing/bears/k4 (100).jpg  
  inflating: dataset/Testing/bears/k4 (101).jpeg  
  inflating: dataset/Testing/bears/k4 (101).jpg  
  inflating: dataset/Testing/bears/k4 (102).jpeg  
  inflating: dataset/Testing/bears/k4 (102).jpg  
  inflating: dataset/Testing/bears/k4 (103).jpeg  
  inflating: dataset/Testing/bears/k4 (104).jpeg  
  inflating: dataset/Testing/bears/k4 (105).jpeg  
  inflating: dataset/Testing/bears/k4 (106).jpeg  
  inflating: dataset/Testing/bears/k4 (107).jpeg  
  inflating: dataset/Testing/bears/k4 (108).jpeg  
  inflating: dataset/Testing/bears/k4 (109).jpeg  
  inflating: dataset/Testing/bears/k4 (110).jpeg  
  inflating: dataset/Testing/bears/k4 (71).jpg  
  inflating: dataset/Testing/bears/k4 (72).jpeg  
  inflating: dataset/Testing/bears/k4 (72).jpg  
  inflating: dataset/Testing/bears/k4 (73).jpeg  
  inflating: dataset/Testing/bears/k4 (73).jp

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
trainpath = '/content/dataset/Training'
testpath = '/content/dataset/Testing'

In [10]:
train_datagen = ImageDataGenerator(rescale = 1./255,zoom_range= 0.2,shear_range= 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255)

when we rescale the images to 0 to 1 from 0 to 255,the training is done faster

In [11]:
train = train_datagen.flow_from_directory(trainpath,target_size =(224,224),batch_size = 16)
test = test_datagen.flow_from_directory(testpath,target_size =(224,224),batch_size = 16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In general the shape of input for vgg16 should be 224,224

VGG16

In [12]:
#import model building libraries
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten


In [13]:
conv_base=VGG16(weights='imagenet',
                include_top=False,
                input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [14]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
model=Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(4,activation="softmax"))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21,138,500
Trainable params: 21,138,500
Non-trainable params: 0
_________________________________________________________________


In [17]:
#Let us freeze the conv base
conv_base.trainable=False


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21,138,500
Trainable params: 6,423,812
Non-trainable params: 14,714,688
_________________________________________________________________


In [18]:
21138500-14714688

6423812

Compile the model

In [19]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics =['accuracy'])

In [20]:
model.fit(train,epochs=2,validation_data=test)

Epoch 1/2
78/78 [==============================] - 1114s 14s/step - loss: 1.1654 - accuracy: 0.7254 - val_loss: 0.0671 - val_accuracy: 0.9847
Epoch 2/2
78/78 [==============================] - 1119s 14s/step - loss: 0.1306 - accuracy: 0.9620 - val_loss: 0.0192 - val_accuracy: 0.9969


steps per epoch=no.of samples in your trainingset/batch size
1238/16=77.35=77

Then save the model and test it

**Resnet 50**

In [21]:
#import model building libraries
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

In [24]:
ResNet_base=ResNet50(weights='imagenet',
                include_top=False,
                input_shape=(224,224,3))

In [25]:
ResNet_base.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [27]:
model1=Sequential()

model1.add(ResNet_base)
model1.add(Flatten())
model1.add(Dense(256,activation="relu"))
model1.add(Dense(4,activation="softmax"))

In [29]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 100352)            0         
                                                                 
 dense_4 (Dense)             (None, 256)               25690368  
                                                                 
 dense_5 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 49,279,108
Trainable params: 49,225,988
Non-trainable params: 53,120
_________________________________________________________________


In [30]:
#Let us freeze the conv base
conv_base.trainable=False

In [31]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics =['accuracy'])
model.fit(train,epochs=2,validation_data=test)

Epoch 1/2
78/78 [==============================] - 416s 5s/step - loss: 3.3382 - accuracy: 0.3990 - val_loss: 0.9939 - val_accuracy: 0.6074
Epoch 2/2
78/78 [==============================] - 396s 5s/step - loss: 1.3312 - accuracy: 0.4960 - val_loss: 1.4741 - val_accuracy: 0.4141
